# Double Variable Polynomial Regression

## Install and Import Dependencies

In [2]:
%pip install numpy pandas matplotlib

Note: you may need to restart the kernel to use updated packages.


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Load Data

In [5]:
df = pd.read_csv('../DoubleVariablePolynomialRegression.ipynb/data/Fish.csv')

In [6]:
df.head()

,Height,Width,Weight
0,11.5200,4.0200,242.0
1,12.4800,4.3056,290.0
2,12.3778,4.6961,340.0
3,12.7300,4.4555,363.0
4,12.4440,5.1340,430.0


## Note: No NAN Values

In [7]:
df.isna().sum()

Height    0
Width     0
Weight    0
dtype: int64

## Normalize the Features

In [9]:
df['Height'] = (df['Height']-np.mean(df['Height']))/np.std(df['Height'])
df['Width'] = (df['Width']-np.mean(df['Width']))/np.std(df['Width'])
df.head()

,Height,Width,Weight
0,0.596579,-0.236529,242.0
1,0.821261,-0.066579,290.0
2,0.797341,0.165793,340.0
3,0.879771,0.022621,363.0
4,0.812835,0.426371,430.0


## Generate Random Test and Train Splits

In [10]:
seed = 420
train_fraction = 0.8
train = df.sample(frac=train_fraction, random_state=seed)
test = df.drop(train.index)

## Polynomial Regression Model

In [139]:
class PolynomialRegressionModel:
    def __init__(self, i, j, q, lmbda):
        """
        Polynomial Regression Model for some particular degree.
        """
        self.loss = None
        self.q = q
        self.lmbda = lmbda
        self.i = i
        self.j = j
        self.weights = np.random.rand(i+1, j+1)

    def calculate_loss(self, X_i, t_i):
        # print('inside self.calculate_loss()')
        assert type(X_i[0]) == np.float64 and type(t_i) == np.float64 and type(X_i[1]) == np.float64, "Types are not matching. Check!"
        result = np.zeros_like(self.weights)
        a = X_i[0]
        b = X_i[1]
        t = t_i
        prediction = self.predict([(a, b)])
        # print('predicted:')
        # print(prediction)
        # print('expected')
        # print(target)

        # tmp = np.sum(self.weights * np.power(a, np.arange(self.i + 1))[:, np.newaxis] * np.power(b, np.arange(self.j + 1)), axis=(0, 1))

        exponents_a = np.arange(self.i + 1)[:, np.newaxis]
        exponents_b = np.arange(self.j + 1)
        result = -1*np.outer(np.power(a, exponents_a), np.power(b, exponents_b)) * (t - prediction)

        # print('loss: ')
        # print(result)
        return result

    def fit(self, X_train, y_train, lr=0.01, epochs=500, batch_size=20):
        """
        Fit the polynomial regression model using Batch Gradient Descent.

        Parameters:
        X_train: Input Feature variables.
        y_train: Target Variable
        lr: Learning Rate for Gradient Descent
        epochs: No of Epochs to train

        Returns:
        NA
        """
        print('Starting Training.....')
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        # print(X_train.head())
        for epoch in range(epochs):
            count = 0
            loss = np.zeros_like(self.weights)
            # print(X_train.shape[0])
            for i in range(X_train.shape[0]):
                # print('sample')
                # print(X_train[i][0])
                # print(X_train[i][1])
                # print(y_train[i])
                X_i = (X_train[i][0],X_train[i][1])
                t_i =  y_train[i]

                if count%batch_size == 0:
                    loss /= batch_size
                    # print('loss: ')
                    # print(loss)
                    self.weights += (self.lmbda*self.q//2)*(np.abs(self.weights))
                    self.weights -= lr*loss
                    loss = np.zeros_like(self.weights)
                else:
                    loss += self.calculate_loss(X_i, t_i)
            
                count+=1
            if epoch%(epochs/10) == 0:
                print(f"epoch: {epoch}")
                print(f"Error: {self.calculate_error(X_train, y_train)}")

        return

    def calculate_error(self, X_test, y_test):
        """
        Find the error of the model on some data.

        Parameters:
        X_test: The sample Input Feature.
        y_test: The sample Target Feature.

        Returns:
        A float value that is the MSE b/w the predicted outputs and the target outputs.
        """
        predictions = self.predict(X_test)
        mse = np.mean(
            (predictions-y_test)**2
        )
        return mse

    def predict(self, X_test):
        """
        Make Predictions using the trained model.

        Parameters:
        X_test: The sample Input Features.

        Returns:
        A numpy Array with the predicted target variable value for each of the samples having
        same dimensions as X_test.
        """
        result = [] 
        for sample in X_test:
            assert type(sample[0]) == np.float64 and type(sample[1]) == np.float64, "Variable doesn't have the required type!"
            a = sample[0]
            b = sample[1]
            tmp = np.sum(self.weights * np.power(a, np.arange(self.i + 1))[:, np.newaxis] * np.power(b, np.arange(self.j + 1)), axis=(0, 1))           
            result.append(tmp)
        return np.array(result)

## Skip

In [125]:
train.head(1)

,Height,Width,Weight
25,1.72935,0.973353,725.0


In [149]:
model = PolynomialRegressionModel(i=2, j=2, q=1, lmbda=0.5)
model.fit(train.drop(['Weight'], axis=1), train['Weight'], lr=0.1, epochs=100)

Starting Training.....
epoch: 0
Error: 45664.56668799255
epoch: 10
Error: 21475.425890441064
epoch: 20
Error: 17777.177604889595
epoch: 30
Error: 16315.139481333717
epoch: 40
Error: 15583.466945549464
epoch: 50
Error: 15144.88876973703
epoch: 60
Error: 14844.663905826492
epoch: 70
Error: 14620.320641726932
epoch: 80
Error: 14443.106367437658
epoch: 90
Error: 14298.019102002194


## Testing

In [165]:
test.drop(['Weight'], axis=1)

,Height,Width
5,1.083951,0.303431
9,1.230041,0.322473
10,1.238514,0.408638
23,1.520723,1.019411
28,1.853299,1.059756
31,2.132841,1.115454
35,-1.128978,-1.279079
37,-0.794670,-0.900381
47,-0.601725,-0.367443
52,-0.022045,0.047197


In [170]:
y_pred = model.predict(list(np.array(test.drop(['Weight'], axis=1))))

In [171]:
y_pred

array([ 474.26997498,  510.64691242,  535.9567227 ,  757.96667186,
        844.83460089,  936.65375086,   37.14566983,   73.25369793,
        171.39044908,  287.96679305,  803.89971268,   86.37040986,
         -8.1353447 ,   50.99947016,   69.9692975 ,  133.24927798,
        160.3711235 ,  225.99305442,  326.82587578, 1234.66898376,
        770.61520881, 1051.37330536,  864.0998368 , 1255.01535491,
        298.60836825,  317.34692594,  260.46643182,  297.46231857,
        454.83120018,  727.53666158,   22.9216226 ,    8.46361698])

In [174]:
true = np.array(test['Weight'])
true

array([ 450. ,  500. ,  475. ,  680. ,  850. ,  955. ,   40. ,   78. ,
        160. ,  290. ,  800. ,   90. ,    5.9,   51.5,   70. ,  130. ,
        170. ,  218. ,  300. ,  840. ,  690. , 1015. , 1100. , 1000. ,
        300. ,  430. ,  456. ,  510. ,  540. , 1600. ,   13.4,   19.9])

In [175]:
correlation_matrix = np.corrcoef(y_pred, true)

In [180]:
%pip install scipy
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/8f/87/5969092159207f583481ad80a03f09e2d4af1ebd197f4530ca4e906c947e/scikit_learn-1.3.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for joblib>=1.1.1 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for threadpoolctl>=2.0.0 from https://files.pythonhosted.org/packages/81/12/fd4dea011af9d69e1cad05c75f3f7202cdcbeac9b712eea58ca779a72865/threadpoolctl-3.2.0-py3-none-any.whl.metadata
Using cached scikit_learn-1.3.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.9 MB)
Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)
Note: you may need to restart the kernel to use up

In [181]:
from sklearn.metrics import r2_score
r2_score(y_pred, true)

0.735245934624742